In [1]:
from importlib import reload
# Tips to me:
# Use Python 3.11.6
# That's in the location /opt/homebrew/bin/python3

# Libraries
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import dataset_creator as dataset
from IPython.display import Audio
from Nets import complicatedNet
from torch.nn import functional as F
import models
import trainer
from dotenv import load_dotenv

import train_validate

load_dotenv()

True

In [2]:
reload(dataset)
all_set = dataset.AudioDatasetAt("datasets/clipses", limit=20000)
trainset, testset, validationset = dataset.split_dataset(all_set, [0.9, 0.05, 0.1])
# testset = dataset.AudioDatasetAt("datasets/clips/test",limit=200)
# trainset = dataset.AudioDatasetAt("datasets/clips/train", limit=10000)
# validationset = dataset.AudioDatasetAt("datasets/clips/validation", limit=500)


In [5]:
reload(dataset)
# Check Loaders
from torch.utils.data import DataLoader
# trainloader = DataLoader(trainset, 32, shuffle=True, collate_fn=dataset.my_collater)
# validationloader = DataLoader(validationset, 8, shuffle=True, collate_fn=dataset.my_collater)
# testloader = DataLoader(testset, 8, shuffle=True, collate_fn=dataset.my_collater)
trainloader = DataLoader(trainset, 32, shuffle=True)
validationloader = DataLoader(validationset, 8, shuffle=True)
testloader = DataLoader(testset, 8, shuffle=True)

In [6]:
# Getting a random data
wf = next(testloader._get_iterator())
inp = wf

print(inp.shape)
Audio(inp[0], rate=32000)

torch.Size([8, 1, 96000])


In [7]:
# Instantiating a model
reload(complicatedNet)
reload(models)
from torch.optim import SGD

encoder = complicatedNet.Encoder(4)
quantizer = torch.nn.Identity()
decoder = complicatedNet.Decoder(4)
my_model = models.allNet(
    encoder=encoder, quantizer=quantizer, decoder=decoder
)
device = 'mps'
my_model.to(device)
dummydata = torch.rand([np.random.randint(21),1,256]).to(device)
my_model.eval()

my_model(dummydata)

lossfn = complicatedNet.LossFunction().to('cpu')
optimizer = SGD(my_model.parameters(), lr=0.1, weight_decay=0.97)

print(my_model)

/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


allNet(
  (encoder): Encoder(
    (encoder): Sequential(
      (0): Down(
        (InConv): Conv1d(1, 4, kernel_size=(1,), stride=(1,))
        (Conv1): ConvolutionalStack(
          (ELU1): ELU(alpha=1.0)
          (Conv1): LazyCausalConv1d(
            (lazyLayer): Conv1d(4, 4, kernel_size=(11,), stride=(1,))
          )
          (BNorm1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Conv2): ConvolutionalStack(
          (ELU1): ELU(alpha=1.0)
          (Conv1): LazyCausalConv1d(
            (lazyLayer): Conv1d(4, 4, kernel_size=(11,), stride=(1,), dilation=(2,))
          )
          (BNorm1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Conv3): ConvolutionalStack(
          (ELU1): ELU(alpha=1.0)
          (Conv1): LazyCausalConv1d(
            (lazyLayer): Conv1d(4, 4, kernel_size=(11,), stride=(1,), dilation=(4,))
          )
          (BNorm1): BatchNorm1d(4, eps=1e-05, mom

In [8]:
x = inp.to(device)
output = my_model(    x
    )[0].cpu().squeeze().unsqueeze(0).detach().numpy()

print(x.shape)
print(output.shape)
print("Before Training, Model is :")
Audio(output, rate=32000)


torch.Size([8, 1, 96000])
(1, 96000)
Before Training, Model is :


In [9]:
reload(trainer)
time, loss = trainer.train_one_epoch(my_model, lossfn, optimizer, trainloader, True)

Training one epoch
Trained on batch 1


RuntimeError: MPS backend out of memory (MPS allocated: 6.97 GB, other allocations: 10.86 GB, max allowed: 18.13 GB). Tried to allocate 375.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [12]:
statistics = {"time": [], "train_losses":[]}

In [4]:
# statistics["time"].append(time)
# statistics["time"].append(loss)
trainer.save_model(my_model, "Complicated-Net-another", statistics)

NameError: name 'my_model' is not defined

In [13]:
# Big Thing
reload(train_validate)

train_validate.train_and_validate_simul(my_model, lossfn, optimizer, trainloader, validationloader, True)

Training for an epoch


RuntimeError: MPS backend out of memory (MPS allocated: 10.29 GB, other allocations: 7.79 GB, max allowed: 18.13 GB). Tried to allocate 585.94 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [20]:
x = inp.to(device)
output = my_model(    x
    )[0].cpu().squeeze().unsqueeze(0).detach().numpy()

print(x.shape)
print(output.shape)
print("After Training, Model is :")
Audio(output, rate=32000)

torch.Size([18, 1, 96000])
(1, 96000)
After Training, Model is :


In [ ]:
idx = 0
size = 13
remaining = lambda : size - idx
idx, remaining()

In [ ]:
idx += 2
idx, remaining()

In [ ]:
spectrogramer1 = torchaudio.transforms.MelSpectrogram(32000)
spectrogramer2 = torchaudio.transforms.MelSpectrogram(32000)

In [ ]:
image1 = spectrogramer1(inp).cpu().detach()
image2 = spectrogramer1(out).cpu().detach()
image1 = F.interpolate(image1, scale_factor=.1, mode='bilinear').squeeze()
image2 = F.interpolate(image2, scale_factor=.1, mode='bilinear').squeeze()
# image1 = F.interpolate(image1, mode='linear', scale_factor=10).squeeze()
image1.shape, image2.shape

In [ ]:
plt.figure(figsize=(20,7))
plt.figure(211)
plt.imshow(image1)
plt.figure(212)
plt.imshow(image2)